In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
from tensorflow import keras
warnings.filterwarnings("ignore")

In [ ]:
labels = pd.read_csv('../input/weight-by-photo/data.csv').drop("Unnamed: 0",axis=1)
train_datagen = ImageDataGenerator(rescale=1./255)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='../input/weight-by-photo/photo/',
        x_col='filename',
        y_col='weight',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345)

In [ ]:
labels.info()

In [ ]:
labels.hist(bins=100)

In [ ]:
features, target = next(train_gen_flow)

fig = plt.figure(figsize=(10,10))
for i in range(12):
    fig.add_subplot(4, 4, i+1)
    plt.imshow(features[i])
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

In [ ]:
def load_train():
    labels = pd.read_csv('../input/weight-by-photo/data.csv').drop("Unnamed: 0",axis=1)
    datagen = ImageDataGenerator(validation_split=0.1, vertical_flip=True,horizontal_flip=True,width_shift_range=0.2,height_shift_range=0.2,rotation_range=90, rescale=1./255)
    train_gen_flow = datagen.flow_from_dataframe(
        dataframe=labels,
        directory='../input/weight-by-photo/photo/',
        x_col='filename',
        y_col='weight',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='training',
        seed=12345)

    return train_gen_flow

def load_test():
    labels = pd.read_csv('../input/weight-by-photo/data.csv').drop("Unnamed: 0",axis=1)
    datagen = ImageDataGenerator(validation_split=0.1, rescale=1./255)
    test_gen_flow = datagen.flow_from_dataframe(
        dataframe=labels,
        directory='../input/weight-by-photo/photo/',
        x_col='filename',
        y_col='weight',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='validation',
        seed=12345)

    return test_gen_flow


def create_model():

    backbone = ResNet50(input_shape=(224,224,3),
                    weights='imagenet', 
                    include_top=False)
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu'))

    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='mean_squared_error',
                  metrics=['mae'])


    return model


def train_model(model, train_data, test_data, batch_size=None, epochs=100,
                steps_per_epoch=None, validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size, 
              epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)

    return model

In [ ]:
train=load_train()
test=load_test()
model=create_model()
model1=train_model(model,train,test)

In [ ]:
model1.save('model1.h5')

In [4]:
model_loaded = keras.models.load_model('model1.h5')

In [6]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (224, 224, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

image = load('2.jpg')
prediction=model_loaded.predict(image)[0][0]
print("Your weight: {:.2f}".format(prediction))

Your weight: 76.92
